# Feature Selection and Ensemble Methods



Importing the neccesary modules 

In [1]:
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score, precision_score, recall_score, f1_score
import time

#import river, an online learning library
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE
from river import feature_selection
from river import stats
from river import imblearn
from river import preprocessing
from river import evaluate
from river import metrics

Loading the BOT-IOT dataset

In [ ]:
# total_rows = sum(1 for _ in open('IoT Network Intrusion Dataset.csv'))  # Replace with your file path

# Load only 10% of the dataset
df = pd.read_csv('IoT Network Intrusion Dataset.csv')
# , nrows=int(total_rows * 0.25)

print(df)

In [3]:
df.dtypes

Flow_ID      object
Src_IP       object
Src_Port      int64
Dst_IP       object
Dst_Port      int64
             ...   
Idle_Max    float64
Idle_Min    float64
Label        object
Cat          object
Sub_Cat      object
Length: 86, dtype: object

In [4]:
df.dtypes.value_counts()

float64    45
int64      34
object      7
dtype: int64

In [5]:
object_columns = df.select_dtypes(include=['object'])
print(object_columns)

                                          Flow_ID           Src_IP  \
0        192.168.0.13-192.168.0.16-10000-10101-17     192.168.0.13   
1         192.168.0.13-222.160.179.132-554-2179-6  222.160.179.132   
2          192.168.0.13-192.168.0.16-9020-52727-6     192.168.0.16   
3          192.168.0.13-192.168.0.16-9020-52964-6     192.168.0.16   
4       192.168.0.1-239.255.255.250-36763-1900-17      192.168.0.1   
...                                           ...              ...   
625778   192.168.0.24-210.89.164.90-56112-8043-17     192.168.0.24   
625779    192.168.0.13-222.131.171.244-554-4570-6  222.131.171.244   
625780     192.168.0.13-192.168.0.16-9020-52739-6     192.168.0.16   
625781     192.168.0.13-192.168.0.16-9020-49784-6     192.168.0.13   
625782   192.168.0.13-192.168.0.16-10000-10101-17     192.168.0.13   

                 Dst_IP               Timestamp    Label     Cat  \
0          192.168.0.16  25/07/2019 03:25:53 AM  Anomaly   Mirai   
1          192.168.0.13

Encoding all the column values that has object datatype to numeric values.

In [6]:
from sklearn.preprocessing import LabelEncoder
Flow_ID_types = df["Flow_ID"].values
encoder = LabelEncoder()
# use LabelEncoder to encode the threat types in numeric values
flow_ID = encoder.fit_transform(Flow_ID_types)
print("Shape of target vector : ",flow_ID.shape)

Shape of target vector :  (625783,)


In [7]:
flow_ID

array([12446, 22760, 12691, ..., 12695, 12682, 12446])

In [8]:
from sklearn.preprocessing import LabelEncoder
Src_IP_types = df["Src_IP"].values
encoder = LabelEncoder()
# use LabelEncoder to encode the threat types in numeric values
src_IP = encoder.fit_transform(Src_IP_types)
print("Shape of target vector : ",src_IP.shape)

Shape of target vector :  (625783,)


In [9]:
src_IP

array([25883, 34617, 25886, ..., 25886, 25883, 25883])

In [10]:
from sklearn.preprocessing import LabelEncoder
Dst_IP_types = df["Dst_IP"].values
encoder = LabelEncoder()
# use LabelEncoder to encode the threat types in numeric values
dst_IP = encoder.fit_transform(Dst_IP_types)
print("Shape of target vector : ",dst_IP.shape)

Shape of target vector :  (625783,)


In [11]:
dst_IP

array([203, 200, 200, ..., 200, 203, 203])

In [12]:
from sklearn.preprocessing import LabelEncoder
Timestamp_types = df["Timestamp"].values
encoder = LabelEncoder()
# use LabelEncoder to encode the threat types in numeric values
timestamp = encoder.fit_transform(Timestamp_types)
print("Shape of target vector : ",timestamp)

Shape of target vector :  [3496 3664 2082 ... 2285 2872 3453]


In [13]:
timestamp

array([3496, 3664, 2082, ..., 2285, 2872, 3453])

In [14]:
from sklearn.preprocessing import LabelEncoder
Label_types = df["Label"].values
encoder = LabelEncoder()
# use LabelEncoder to encode the threat types in numeric values
label = encoder.fit_transform(Label_types)
print("Shape of target vector : ",label)

Shape of target vector :  [0 0 0 ... 0 1 0]


In [15]:
from sklearn.preprocessing import LabelEncoder
Cat_types = df["Cat"].values
encoder = LabelEncoder()
# use LabelEncoder to encode the threat types in numeric values
cat = encoder.fit_transform(Cat_types)
print("Shape of target vector : ",cat)

Shape of target vector :  [2 0 4 ... 4 3 2]


In [16]:
from sklearn.preprocessing import LabelEncoder
Sub_Cat_types = df["Sub_Cat"].values
encoder = LabelEncoder()
# use LabelEncoder to encode the threat types in numeric values
sub_cat = encoder.fit_transform(Sub_Cat_types)
print("Shape of target vector : ",sub_cat)

Shape of target vector :  [2 0 8 ... 8 6 5]


In [17]:
from sklearn.preprocessing import MinMaxScaler

large_finite_value = 1e10  # Adjust the value as needed
column_name = "Flow_Byts/s"  # Replace with the name of the column you want to modify

df[column_name] = df[column_name].replace([np.inf, -np.inf], large_finite_value)

# Apply Min-Max Scaling to the specific column
scaler = MinMaxScaler()
df[[column_name]] = scaler.fit_transform(df[[column_name]])

In [18]:
from sklearn.preprocessing import MinMaxScaler

large_finite_value = 1e10  # Adjust the value as needed
column_name = "Flow_Pkts/s"  # Replace with the name of the column you want to modify

df[column_name] = df[column_name].replace([np.inf, -np.inf], large_finite_value)

# Apply Min-Max Scaling to the specific column
scaler = MinMaxScaler()
df[[column_name]] = scaler.fit_transform(df[[column_name]])

In [19]:
columns_to_drop = ["Flow_ID","Src_IP","Dst_IP","Timestamp","Label","Cat","Sub_Cat"]

# Drop the specified columns
df = df.drop(columns=columns_to_drop)

# Print the modified DataFrame
print(df)


        Src_Port  Dst_Port  Protocol  Flow_Duration  Tot_Fwd_Pkts  \
0          10000     10101        17             75             1   
1           2179       554         6           5310             1   
2          52727      9020         6            141             0   
3          52964      9020         6            151             0   
4          36763      1900        17            153             2   
...          ...       ...       ...            ...           ...   
625778     56112      8043        17            277             1   
625779      4570       554         6           1658             0   
625780     52739      9020         6             77             1   
625781      9020     49784         6            240             2   
625782     10000     10101        17            198             2   

        Tot_Bwd_Pkts  TotLen_Fwd_Pkts  TotLen_Bwd_Pkts  Fwd_Pkt_Len_Max  \
0                  1            982.0           1430.0            982.0   
1                  2 

In [20]:
df.columns

Index(['Src_Port', 'Dst_Port', 'Protocol', 'Flow_Duration', 'Tot_Fwd_Pkts',
       'Tot_Bwd_Pkts', 'TotLen_Fwd_Pkts', 'TotLen_Bwd_Pkts', 'Fwd_Pkt_Len_Max',
       'Fwd_Pkt_Len_Min', 'Fwd_Pkt_Len_Mean', 'Fwd_Pkt_Len_Std',
       'Bwd_Pkt_Len_Max', 'Bwd_Pkt_Len_Min', 'Bwd_Pkt_Len_Mean',
       'Bwd_Pkt_Len_Std', 'Flow_Byts/s', 'Flow_Pkts/s', 'Flow_IAT_Mean',
       'Flow_IAT_Std', 'Flow_IAT_Max', 'Flow_IAT_Min', 'Fwd_IAT_Tot',
       'Fwd_IAT_Mean', 'Fwd_IAT_Std', 'Fwd_IAT_Max', 'Fwd_IAT_Min',
       'Bwd_IAT_Tot', 'Bwd_IAT_Mean', 'Bwd_IAT_Std', 'Bwd_IAT_Max',
       'Bwd_IAT_Min', 'Fwd_PSH_Flags', 'Bwd_PSH_Flags', 'Fwd_URG_Flags',
       'Bwd_URG_Flags', 'Fwd_Header_Len', 'Bwd_Header_Len', 'Fwd_Pkts/s',
       'Bwd_Pkts/s', 'Pkt_Len_Min', 'Pkt_Len_Max', 'Pkt_Len_Mean',
       'Pkt_Len_Std', 'Pkt_Len_Var', 'FIN_Flag_Cnt', 'SYN_Flag_Cnt',
       'RST_Flag_Cnt', 'PSH_Flag_Cnt', 'ACK_Flag_Cnt', 'URG_Flag_Cnt',
       'CWE_Flag_Count', 'ECE_Flag_Cnt', 'Down/Up_Ratio', 'Pkt_Size_Avg',
       

In [21]:
x_col = ['Src_Port', 'Dst_Port', 'Protocol', 'Flow_Duration', 'Tot_Fwd_Pkts',
       'Tot_Bwd_Pkts', 'TotLen_Fwd_Pkts', 'TotLen_Bwd_Pkts', 'Fwd_Pkt_Len_Max',
       'Fwd_Pkt_Len_Min', 'Fwd_Pkt_Len_Mean', 'Fwd_Pkt_Len_Std',
       'Bwd_Pkt_Len_Max', 'Bwd_Pkt_Len_Min', 'Bwd_Pkt_Len_Mean',
       'Bwd_Pkt_Len_Std', 'Flow_Byts/s', 'Flow_Pkts/s', 'Flow_IAT_Mean',
       'Flow_IAT_Std', 'Flow_IAT_Max', 'Flow_IAT_Min', 'Fwd_IAT_Tot',
       'Fwd_IAT_Mean', 'Fwd_IAT_Std', 'Fwd_IAT_Max', 'Fwd_IAT_Min',
       'Bwd_IAT_Tot', 'Bwd_IAT_Mean', 'Bwd_IAT_Std', 'Bwd_IAT_Max',
       'Bwd_IAT_Min', 'Fwd_PSH_Flags', 'Bwd_PSH_Flags', 'Fwd_URG_Flags',
       'Bwd_URG_Flags', 'Fwd_Header_Len', 'Bwd_Header_Len', 'Fwd_Pkts/s',
       'Bwd_Pkts/s', 'Pkt_Len_Min', 'Pkt_Len_Max', 'Pkt_Len_Mean',
       'Pkt_Len_Std', 'Pkt_Len_Var', 'FIN_Flag_Cnt', 'SYN_Flag_Cnt',
       'RST_Flag_Cnt', 'PSH_Flag_Cnt', 'ACK_Flag_Cnt', 'URG_Flag_Cnt',
       'CWE_Flag_Count', 'ECE_Flag_Cnt', 'Down/Up_Ratio', 'Pkt_Size_Avg',
       'Fwd_Seg_Size_Avg', 'Bwd_Seg_Size_Avg', 'Fwd_Byts/b_Avg',
       'Fwd_Pkts/b_Avg', 'Fwd_Blk_Rate_Avg', 'Bwd_Byts/b_Avg',
       'Bwd_Pkts/b_Avg', 'Bwd_Blk_Rate_Avg', 'Subflow_Fwd_Pkts',
       'Subflow_Fwd_Byts', 'Subflow_Bwd_Pkts', 'Subflow_Bwd_Byts',
       'Init_Fwd_Win_Byts', 'Init_Bwd_Win_Byts', 'Fwd_Act_Data_Pkts',
       'Fwd_Seg_Size_Min', 'Active_Mean', 'Active_Std', 'Active_Max',
       'Active_Min', 'Idle_Mean', 'Idle_Std', 'Idle_Max', 'Idle_Min']

x = df[x_col].copy()

x.insert(79,"Src_IP",src_IP)
x.insert(80,"Dst_IP",dst_IP)
x.insert(81,"Timestamp",timestamp)
# x.insert(82,"Label",label)
x.insert(82,"Cat",cat)
x.insert(83,"Sub_Cat",sub_cat)
x.insert(84,"Flow_ID",flow_ID)

x.shape

(625783, 85)

In [22]:
x.columns

Index(['Src_Port', 'Dst_Port', 'Protocol', 'Flow_Duration', 'Tot_Fwd_Pkts',
       'Tot_Bwd_Pkts', 'TotLen_Fwd_Pkts', 'TotLen_Bwd_Pkts', 'Fwd_Pkt_Len_Max',
       'Fwd_Pkt_Len_Min', 'Fwd_Pkt_Len_Mean', 'Fwd_Pkt_Len_Std',
       'Bwd_Pkt_Len_Max', 'Bwd_Pkt_Len_Min', 'Bwd_Pkt_Len_Mean',
       'Bwd_Pkt_Len_Std', 'Flow_Byts/s', 'Flow_Pkts/s', 'Flow_IAT_Mean',
       'Flow_IAT_Std', 'Flow_IAT_Max', 'Flow_IAT_Min', 'Fwd_IAT_Tot',
       'Fwd_IAT_Mean', 'Fwd_IAT_Std', 'Fwd_IAT_Max', 'Fwd_IAT_Min',
       'Bwd_IAT_Tot', 'Bwd_IAT_Mean', 'Bwd_IAT_Std', 'Bwd_IAT_Max',
       'Bwd_IAT_Min', 'Fwd_PSH_Flags', 'Bwd_PSH_Flags', 'Fwd_URG_Flags',
       'Bwd_URG_Flags', 'Fwd_Header_Len', 'Bwd_Header_Len', 'Fwd_Pkts/s',
       'Bwd_Pkts/s', 'Pkt_Len_Min', 'Pkt_Len_Max', 'Pkt_Len_Mean',
       'Pkt_Len_Std', 'Pkt_Len_Var', 'FIN_Flag_Cnt', 'SYN_Flag_Cnt',
       'RST_Flag_Cnt', 'PSH_Flag_Cnt', 'ACK_Flag_Cnt', 'URG_Flag_Cnt',
       'CWE_Flag_Count', 'ECE_Flag_Cnt', 'Down/Up_Ratio', 'Pkt_Size_Avg',
       

In [23]:
x

,Src_Port,Dst_Port,Protocol,Flow_Duration,Tot_Fwd_Pkts,Tot_Bwd_Pkts,TotLen_Fwd_Pkts,TotLen_Bwd_Pkts,Fwd_Pkt_Len_Max,Fwd_Pkt_Len_Min,...,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Src_IP,Dst_IP,Timestamp,Cat,Sub_Cat,Flow_ID
0,10000,10101,17,75,1,1,982.0,1430.0,982.0,982.0,...,75.0,0.000000,75.0,75.0,25883,203,3496,2,2,12446
1,2179,554,6,5310,1,2,0.0,0.0,0.0,0.0,...,2655.0,2261.327486,4254.0,1056.0,34617,200,3664,0,0,22760
2,52727,9020,6,141,0,3,0.0,2806.0,0.0,0.0,...,70.5,0.707107,71.0,70.0,25886,200,2082,4,8,12691
3,52964,9020,6,151,0,2,0.0,2776.0,0.0,0.0,...,151.0,0.000000,151.0,151.0,25886,200,791,2,4,12704
4,36763,1900,17,153,2,1,886.0,420.0,452.0,434.0,...,76.5,0.707107,77.0,76.0,25881,317,1040,2,4,611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625778,56112,8043,17,277,1,1,18.0,18.0,18.0,18.0,...,277.0,0.000000,277.0,277.0,25889,233,3443,2,5,62081
625779,4570,554,6,1658,0,2,0.0,0.0,0.0,0.0,...,1658.0,0.000000,1658.0,1658.0,30623,200,3637,0,0,18760
625780,52739,9020,6,77,1,1,0.0,0.0,0.0,0.0,...,77.0,0.000000,77.0,77.0,25886,200,2285,4,8,12695
625781,9020,49784,6,240,2,1,2776.0,1388.0,1388.0,1388.0,...,120.0,7.071068,125.0,115.0,25883,203,2872,3,6,12682


In [24]:
X=x.values
Y = pd.DataFrame()

# Add the "Label" column to the DataFrame
Y["Label"] = label

# Convert the DataFrame to a flattened NumPy array
y= Y.values.flatten().ravel()

Importing Boruta Module and Using RandomForest as the selection classifier. 

In [25]:
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy
# define random forest classifier, with utilising all cores and
# sampling in proportion to y labels
rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)



# define Boruta feature selection method
feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=1)


# find all relevant features - 5 features should be selected
feat_selector.fit(X,y)


Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	85
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	85
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	85
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	85
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	85
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	85
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	85
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	66
Tentative: 	6
Rejected: 	13
Iteration: 	9 / 100
Confirmed: 	66
Tentative: 	6
Rejected: 	13
Iteration: 	10 / 100
Confirmed: 	66
Tentative: 	6
Rejected: 	13
Iteration: 	11 / 100
Confirmed: 	66
Tentative: 	6
Rejected: 	13
Iteration: 	12 / 100
Confirmed: 	68
Tentative: 	4
Rejected: 	13
Iteration: 	13 / 100
Confirmed: 	68
Tentative: 	4
Rejected: 	13
Iteration: 	14 / 100
Confirmed: 	68
Tentative: 	4
Rejected: 	13
Iteration: 	15 / 100
Confirmed: 	68
Tentative: 	4
Rejected: 	13
Iteration: 	16 / 100
Confirmed: 	68
Tentative: 	4
Reject

BorutaPy(estimator=RandomForestClassifier(class_weight='balanced', max_depth=5,
                                          n_estimators=236, n_jobs=-1,
                                          random_state=RandomState(MT19937) at 0x1BE24482240),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x1BE24482240, verbose=2)

The features remained after the unnecessary features removed by the boruta algorithm are 69

In [26]:
selected_feature_indices = feat_selector.support_

# Get the selected feature names
selected_features = [x_col[i] for i in selected_feature_indices]
print('Selected features:', selected_features)

# Zip my names, ranks, and decisions in a single iterable
feature_ranks = list(zip(x_col, feat_selector.ranking_, feat_selector.support_))
for feat in feature_ranks:
    print('Feature: {:<25} Rank: {},  Keep: {}'.format(feat[0], feat[1], feat[2]))

# Print other relevant information
print("Ranking: ", feat_selector.ranking_)          
print("No. of significant features: ", feat_selector.n_features_)

Selected features: ['Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Src_Port', 'Dst_Port', 'Src_Port', 'Src_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Src_Port', 'Dst_Port', 'Src_Port', 'Dst_Port', 'Dst_Port', 'Src_Port', 'Src_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Src_Port', 'Src_Port', 'Src_Port', 'Src_Port', 'Src_Port', 'Src_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Src_Port', 'Dst_Port', 'Dst_Port', 'Src_Port', 'Dst_Port', 'Src_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Por

C:\Users\Administrator\AppData\Local\Temp\ipykernel_9604\3993736446.py:4: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  selected_features = [x_col[i] for i in selected_feature_indices]


Importing Recursive feature elimination and using randomforest as selection Classifier

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE

# Define the classifier
rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)

rfe_selector = RFE(estimator=rf)

# Fit the RFE feature selector
rfe_selector.fit(X, y)




RFE(estimator=RandomForestClassifier(class_weight='balanced', max_depth=5,
                                     n_jobs=-1))

After removing unnecessary features, 42 features are remained.

In [28]:
selected_feature_indices = rfe_selector.support_

# Get the selected feature names
selected_features1 = [x_col[i] for i in selected_feature_indices]
print('Selected features:', selected_features1)

# Zip my names, ranks, and decisions in a single iterable
feature_ranks1= list(zip(x_col, rfe_selector.ranking_, rfe_selector.support_))
for rfe in feature_ranks1:
    print('Feature: {:<25} Rank: {},  Keep: {}'.format(rfe[0], rfe[1], rfe[2]))

# Print other relevant information
print("Ranking: ", rfe_selector.ranking_)          
print("No. of significant features: ", rfe_selector.n_features_)

Selected features: ['Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Src_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Src_Port', 'Dst_Port', 'Src_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Src_Port', 'Dst_Port', 'Src_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Src_Port', 'Src_Port', 'Src_Port', 'Src_Port', 'Src_Port', 'Dst_Port', 'Src_Port', 'Dst_Port', 'Dst_Port', 'Src_Port', 'Src_Port', 'Src_Port', 'Src_Port', 'Src_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Src_Port', 'Dst_Port', 'Src_Port', 'Src_Port', 'Src_Port', 'Src_Port', 'Src_Port', 'Src_Port', 'Src_Port', 'Dst_Port', 'Src_Port', 'Src_Port', 'Src_Port', 'Src_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Src_Port', 'Src_Port', 'Src_Port', 'Src_Port', 'Src_Port', 'Src_Port', 'Src_Port', 'Dst_Port', 'Src_Port', 'Dst_Port', 'Src_Port', 'Dst_Port', 'Src_Port', 'Src_Port', 'Src_Port', 'Src_Port', 'Src_Port', 'Src_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Port', 'Dst_Por

C:\Users\Administrator\AppData\Local\Temp\ipykernel_9604\3546982211.py:4: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  selected_features1 = [x_col[i] for i in selected_feature_indices]


In [29]:
selected_features1 = np.where(rfe_selector.support_)[0]
selected_features = np.where(feat_selector.support_)[0]

Finding the intersection of features selected by both boruta and Recursive feature elimination

In [30]:
intersection_features = np.intersect1d(selected_features1, selected_features)
print("Intersection of selected features:", intersection_features)
print(len(intersection_features))

Intersection of selected features: [ 0  1  2  3  5  6  7  8 10 12 13 14 16 18 19 20 21 27 29 30 36 37 38 39
 41 49 54 55 56 64 66 68 75 76 77 78 79 80 81 82 83 84]
42


Created different Ensembling methods to find the minimum estimators required and the minimum execution time required.

In [35]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X[:, intersection_features], y, test_size=0.2, random_state=42)
start = time.time()
# Define and train an AdaBoost classifier using the selected features
clf = AdaBoostClassifier(n_estimators=2, random_state=0)
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
end = time.time()

print("Model Accuracy:", accuracy)
print("The time of execution of above program is :", (end-start) * 10**3, "ms")

Model Accuracy: 1.0
The time of execution of above program is : 1972.0876216888428 ms


In [36]:
from sklearn.ensemble import GradientBoostingClassifier
start=time.time()
# Define and train a Gradient Boosting classifier using the selected features
clf = GradientBoostingClassifier(n_estimators=5, learning_rate=0.1, random_state=0)
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
end=time.time()
print("Model Accuracy:", accuracy)
print("The time of execution of above program is :", (end-start) * 10**3, "ms")


Model Accuracy: 1.0
The time of execution of above program is : 6273.15616607666 ms


In [37]:
from sklearn.ensemble import ExtraTreesClassifier


start=time.time()
clf = ExtraTreesClassifier(n_estimators=500, max_depth=None, min_samples_split=2, random_state=0)
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
end=time.time()
print("Model Accuracy:", accuracy)
print("The time of execution of above program is :", (end-start) * 10**3, "ms")

Model Accuracy: 1.0
The time of execution of above program is : 144714.63203430176 ms


In [38]:
from sklearn.ensemble import BaggingClassifier

base_classifier = DecisionTreeClassifier(max_depth=5, random_state=0)
start=time.time()
# Define and train a Bagging classifier using the selected features
clf = BaggingClassifier(base_estimator=base_classifier, n_estimators=1, random_state=0)


clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
end=time.time()
print("Model Accuracy:", accuracy)
print("The time of execution of above program is :", (end-start) * 10**3, "ms")


C:\Users\Administrator\AppData\Roaming\Python\Python310\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Model Accuracy: 1.0
The time of execution of above program is : 777.4147987365723 ms


The Bagging Classifier requires inly one estimator and can execute in less time compared to other ensembling methods.
So, Bagging Classifier is selected as the model for the prediction.